In [1]:
from fastai.tabular.all import *
import pandas as pd
import numpy as np
import pickle

In [2]:
with open(f"cache/training_data.pickle", 'rb') as f:
    data = pickle.load(f)
columns_output = list(map(str, list(data['limits'])))
degrees = data['degrees']
lengths = data['lengths']
degree_length_inputs = []
for degree in degrees:
    for length in lengths:
        degree_length_inputs.append(f"{degree}-{length}")

In [4]:
input_symbols = np.array(data['input_symbols'])
input_symbols_unique = set(data['input_symbols'])
data_input = pd.DataFrame(data=data['input'].transpose(), columns=degree_length_inputs)
#data_input['symbol'] = data['input_symbols']
#data_input['symbol'] = data_input['symbol'].astype('|S')
for symbol in input_symbols_unique:
    data_input[symbol] = np.where(input_symbols == symbol, True, False)

In [5]:
data_input

,1-7,1-10,1-15,1-22,1-33,1-47,1-68,1-100,1-150,2-7,...,ALGOUSDT,ANKRUSDT,BCHUSDT,ATOMUSDT,ARPAUSDT,BTTUSDT,ADAUSDT,COSUSDT,BANDUSDT,BATUSDT
0,0.000232,-0.000088,-0.005389,-0.000519,0.000688,-0.000471,-0.000516,-0.000043,-0.000117,-0.000448,...,False,False,True,False,False,False,False,False,False,False
1,0.000232,-0.000088,-0.005389,-0.000519,0.000688,-0.000471,-0.000516,-0.000043,-0.000117,-0.000448,...,False,False,True,False,False,False,False,False,False,False
2,0.000123,-0.001421,-0.004883,-0.000438,0.000478,-0.002270,-0.000423,-0.000342,0.000092,-0.000484,...,False,False,True,False,False,False,False,False,False,False
3,0.000123,-0.001421,-0.004883,-0.000438,0.000478,-0.002270,-0.000423,-0.000342,0.000092,-0.000484,...,False,False,True,False,False,False,False,False,False,False
4,0.000123,-0.001421,-0.004883,-0.000438,0.000478,-0.002270,-0.000423,-0.000342,0.000092,-0.000484,...,False,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7206722,0.005020,0.001552,0.005392,0.001941,0.008152,0.001325,-0.001536,0.006433,0.009722,-0.001631,...,False,False,False,False,False,False,False,True,False,False
7206723,0.005020,0.001552,0.005392,0.001941,0.008152,0.001325,-0.001536,0.006433,0.009722,-0.001631,...,False,False,False,False,False,False,False,True,False,False
7206724,0.003187,-0.001004,0.000762,0.002721,0.004315,-0.004140,-0.000857,0.006837,0.004735,-0.001467,...,False,False,False,False,False,False,False,True,False,False
7206725,0.003187,-0.001004,0.000762,0.002721,0.004315,-0.004140,-0.000857,0.006837,0.004735,-0.001467,...,False,False,False,False,False,False,False,True,False,False


In [6]:
data_output = pd.DataFrame(data=data['output'].transpose(), columns=columns_output)

In [7]:
df = pd.concat((data_input, data_output.reindex(data_input.index)), axis=1)
df.columns = list(map(str, list(df.columns)))
del data_input
del data_output

In [8]:
cont_names = degree_length_inputs
cat_names = list(input_symbols_unique)
y_names = list(map(str, list(data['limits'])))

In [9]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df))

In [10]:
to = TabularPandas(df, procs=[Categorify], cat_names=cat_names, cont_names=cont_names, y_names=y_names, splits=splits)

In [11]:
dls = to.dataloaders(bs=2**17)

In [12]:
learn = tabular_learner(dls, metrics=rmse)

In [ ]:
learn.fit_one_cycle(40)

epoch,train_loss,valid_loss,_rmse,time


In [ ]:
learn.show_results(max_n=50)

In [ ]:
learn.export('model_all.pickle')